# Run a basic Hermes Hartree-Fock energy calculation on ligand conformers from Auto3d

In this notebook, we'll perform a hermes energy calculation on a ligand derived from a smiles string

In [1]:
import json
import os
import sys
import tarfile

from pdbtools import *
import requests
from datetime import datetime
from pathlib import Path
import py3Dmol

import rush

## 0) Setup

In [2]:
EXPERIMENT = "tengu-py-auto3d_hermes"
LIGAND_SMILES = "CC(=O)OC1=CC=CC=C1C(=O)O "
LIGAND = "ASPRIN"
TAGS = ["qdx", EXPERIMENT, LIGAND]

In [3]:
# |hide
WORK_DIR = Path.home() / "qdx" / EXPERIMENT

if WORK_DIR.exists():
    client = rush.Provider(workspace=WORK_DIR)
    await client.nuke(remote=False)

os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(WORK_DIR / ".rush", exist_ok=True)
import sys

os.chdir(WORK_DIR)

In [ ]:
# Get our client, for calling modules and using the rush API
client = await rush.build_provider_with_functions(batch_tags=TAGS)

In [ ]:
# |hide
client = await rush.build_provider_with_functions(batch_tags=TAGS, restore_by_default=True)

In [ ]:
# |hide
client.workspace = WORK_DIR
client.config_dir = WORK_DIR / ".rush"

## 1) Preparation
We want to convert our raw files into prepared qdxfs, with correct charges and missing residues filled

In [ ]:
help(client.auto3d)

In [ ]:
ligand_path = client.workspace / "asprin.smiles"
with open(ligand_path, "w") as ligand_file:
    print(f"{LIGAND_SMILES} {LIGAND_SMILES}", file=ligand_file)

(conformer_sdf, conformer_qdxf) = await client.auto3d(
    ligand_path, "smi", {"k": 5, "use_gpu": True}, resources={"gpus": 1, "storage": "5", "storage_units": "GB"},
    target="NIX_SSH_3"
)

print(f"{datetime.now().time()} | Running ligand preparation!")

In [ ]:
try:
    await conformer_sdf.download(filename="01_prepared_ligand.sdf")
except FileExistsError:
    # we will raise an error if you try to overwrite an existing file; you can
    # force the file to overwrite by passing an absolute filepath instead
    pass

You should visualize your prepared ligand to spot check any issues

In [ ]:
view = py3Dmol.view()
with open(client.workspace / "objects" / "01_prepared_ligand.sdf", "r") as f:
    view.addModel(f.read(), "sdf")
    view.setStyle({"stick": {}})
    view.zoomTo()
    view.show()

## 2) Quantum energy calculation
Finally, we submit our fragmented protein for quantum energy calculation, with custom configuration.

In [ ]:
help(client.hermes_energy)

In [ ]:
HERMES_RESOURCES = {
        "gpus": 4,
        "storage": 100,
        "storage_units": "MB",
        "walltime": 60 * 4,
}

In [ ]:
(conformer,) = await client.pick_conformer(conformer_qdxf, 0)
conformer_out = await conformer.get()

(hermes_energy,) = await client.hermes_energy(
    conformer,
    {},
    {
        "method": "RestrictedRIMP2",
        "basis": "cc-pVDZ",
        "aux_basis": "cc-pVDZ-RIFIT",
        "force_cartesian_basis_sets": True,
        "standard_orientation": "FullSystem",
    },
    {
    "scf": {
        "max_iter": 50,
        "max_diis_history_length": 12,
        "convergence_metric": "DIIS",
        "convergence_threshold": 0.000001,
        "density_threshold": 1e-10,
        "density_basis_set_projection_fallback_enabled": True,
        },
    "frag": {
        "cutoffs": {"dimer": 22},
        "cutoff_type": "Centroid",
        "level": "Dimer",
        "reference_fragment": len(conformer_out["topology"]["fragments"])
        - 1,
        },
    },
    target="NIX_SSH_3",
    resources=HERMES_RESOURCES,
)

In [ ]:
await hermes_energy.get()